In [1]:
from esfMRI import sliceWindows, align
from nilearn import connectome
import pickle
import math
import os

In [2]:
# 可调节参数
window_length_Second = [120, 150, 180]
sliding_steps = [1]

In [3]:
with open("time_series2.pkl", "rb") as f:
    data = pickle.load(f)

In [4]:
slidingWindows = {}
dFCs = {}
for subid in data:
    slidingWindows[subid] = {}
    dFCs[subid] = {}
    for time in window_length_Second:
        windows_preop = []
        windows_postop = []
        window_length_preop = []
        window_length_postop = []
        slidingWindows[subid]["ses-preop"] = {}
        dFCs[subid]["ses-preop"] = {}
        for run, items in data[subid]["ses-preop"].items():
            preopFrame = math.ceil(time/items["TR"])
            tmp = sliceWindows(items["time_series"], preopFrame, 1)
            slidingWindows[subid]["ses-preop"][run] = tmp
            dFCs[subid]["ses-preop"][run] = connectome.ConnectivityMeasure(kind="correlation").fit_transform(tmp)
            window_length_preop.append(len(tmp))
            windows_preop += tmp
        slidingWindows[subid]["ses-postop"] = {}
        dFCs[subid]["ses-postop"] = {}
        for run, items in data[subid]["ses-postop"].items():
            postopFrame = math.ceil(time/items["TR"])
            tmp = sliceWindows(items["time_series"], postopFrame, 1)
            slidingWindows[subid]["ses-postop"][run] = tmp
            dFCs[subid]["ses-postop"][run] = connectome.ConnectivityMeasure(kind="correlation").fit_transform(tmp)
            window_length_postop.append(len(tmp))
            windows_postop += tmp
        slidingWindows[subid]["ses-preop"]["total"] = windows_preop
        slidingWindows[subid]["ses-postop"]["total"] = windows_postop
        slidingWindows[subid]["length"] = [window_length_preop, window_length_postop]
        dFCs[subid]["length"] = [window_length_preop, window_length_postop]
with open("slidingWindows.pkl", "wb") as f:
    pickle.dump(slidingWindows, f)
with open("dFCs.pkl", "wb") as f:
    pickle.dump(dFCs, f)